# Reshaping data for model
### In this file, we will take the data that has been gathered and harmonized and we will fit it, create scenarios, and save it as a structured array for the model. Since we would like to keep the flexibility with excel, we will also save it in an ODYM compatible format and create a file that can do the reverse: if teh excel file is eddited, so is the array. This will be a separate script

In [1]:
# Load a local copy of the current ODYM branch:
import sys
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from seaborn.palettes import color_palette
import xlrd
import pylab
from copy import deepcopy
import logging as log
import xlwt
import tqdm
import math
from scipy.stats import norm
from tqdm import tqdm
from scipy.optimize import curve_fit
import matplotlib
from logistic import logistic as logistic
mpl_logger = log.getLogger("matplotlib")
mpl_logger.setLevel(log.WARNING)  
# For Ipython Notebook only
### Preamble
# Going to parent path
os.getcwd()
os.chdir("..")
os.chdir("..")

# add ODYM module directory to system path, relative
MainPath = os.path.join(os.getcwd(), 'odym', 'modules')
sys.path.insert(0, MainPath)

# add ODYM module directory to system path, absolute
sys.path.insert(0, os.path.join(os.getcwd(), 'odym', 'modules'))

# Specify path to dynamic stock model and to datafile, relative
DataPath = os.path.join( 'docs', 'files')

# Specify path to dynamic stock model and to datafile, absolute
DataPath = os.path.join(os.getcwd(), 'docs', 'Files')

import ODYM_Classes as msc # import the ODYM class file
import ODYM_Functions as msf # import the ODYM function file
import dynamic_stock_model as dsm # import the dynamic stock model library

# Initialize loggin routine
log_verbosity = eval("log.DEBUG")
log_filename = 'LogFileTest.md'
[Mylog, console_log, file_log] = msf.function_logger(log_filename, os.getcwd(),
                                                     log_verbosity, log_verbosity)
Mylog.info('### 1. - Initialize.')

#Read main script parameters
#Load project-specific config file
ProjectSpecs_ConFile = 'ODYM_Config_Vehicle_System.xlsx'
Model_Configfile     = xlrd.open_workbook(os.path.join(DataPath, ProjectSpecs_ConFile))
ScriptConfig         = {'Model Setting': Model_Configfile.sheet_by_name('Config').cell_value(3,3)} # Dictionary with config parameters
Model_Configsheet    = Model_Configfile.sheet_by_name('Setting_' + ScriptConfig['Model Setting'])

Name_Scenario        = Model_Configsheet.cell_value(3,3)
print(Name_Scenario)

#Read control and selection parameters into dictionary
ScriptConfig         = msf.ParseModelControl(Model_Configsheet,ScriptConfig)

Mylog.info('Read and parse config table, including the model index table, from model config sheet.')
IT_Aspects,IT_Description,IT_Dimension,IT_Classification,IT_Selector,IT_IndexLetter,\
PL_Names,PL_Description,PL_Version,PL_IndexStructure,PL_IndexMatch,PL_IndexLayer,\
PrL_Number,PrL_Name,PrL_Comment,PrL_Type,ScriptConfig = msf.ParseConfigFile(Model_Configsheet,ScriptConfig,Mylog)    

class_filename       = 'ODYM_Classifications_Master_Vehicle_System.xlsx'
Classfile            = xlrd.open_workbook(os.path.join(DataPath,class_filename))
Classsheet           = Classfile.sheet_by_name('MAIN_Table')
MasterClassification = msf.ParseClassificationFile_Main(Classsheet,Mylog)


Mylog.info('Define model classifications and select items for model classifications according to information provided by config file.')
ModelClassification  = {} # Dict of model classifications
for m in range(0,len(IT_Aspects)):
    ModelClassification[IT_Aspects[m]] = deepcopy(MasterClassification[IT_Classification[m]])
    EvalString = msf.EvalItemSelectString(IT_Selector[m],len(ModelClassification[IT_Aspects[m]].Items))
    if EvalString.find(':') > -1: # range of items is taken
        RangeStart = int(EvalString[0:EvalString.find(':')])
        RangeStop  = int(EvalString[EvalString.find(':')+1::])
        ModelClassification[IT_Aspects[m]].Items = ModelClassification[IT_Aspects[m]].Items[RangeStart:RangeStop]           
    elif EvalString.find('[') > -1: # selected items are taken
        ModelClassification[IT_Aspects[m]].Items = [ModelClassification[IT_Aspects[m]].Items[i] for i in eval(EvalString)]
    elif EvalString == 'all':
        None
    else:
        Mylog.error('Item select error for aspect ' + IT_Aspects[m] + ' were found in datafile.')
        break

# Define model index table and parameter dictionary
Mylog.info('### 2.2 - Define model index table and parameter dictionary')
Model_Time_Start = int(min(ModelClassification['Time'].Items))
Model_Time_End   = int(max(ModelClassification['Time'].Items))
Model_Duration   = Model_Time_End - Model_Time_Start + 1

Mylog.info('Define index table dataframe.')
IndexTable = pd.DataFrame({'Aspect'        : IT_Aspects,  # 'Time' and 'Element' must be present!
                           'Description'   : IT_Description,
                           'Dimension'     : IT_Dimension,
                           'Classification': [ModelClassification[Aspect] for Aspect in IT_Aspects],
                           'IndexLetter'   : IT_IndexLetter})  # Unique one letter (upper or lower case) indices to be used later for calculations.

# Default indexing of IndexTable, other indices are produced on the fly
IndexTable.set_index('Aspect', inplace=True)

# Add indexSize to IndexTable:
IndexTable['IndexSize'] = pd.Series([len(IndexTable.Classification[i].Items) for i in range(0, len(IndexTable.IndexLetter))],
                                    index=IndexTable.index)

# list of the classifications used for each indexletter
IndexTable_ClassificationNames = [IndexTable.Classification[i].Name for i in range(0, len(IndexTable.IndexLetter))]


# Define dimension sizes
Nt = len(IndexTable.Classification[IndexTable.index.get_loc('Time')].Items)
Nc = len(IndexTable.Classification[IndexTable.index.get_loc('Age-cohort')].Items)
Ng = len(IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items)
Ne = len(IndexTable.Classification[IndexTable.index.get_loc('Element')].Items)
Nb = len(IndexTable.Classification[IndexTable.index.get_loc('Battery_Chemistry')].Items)
Ns = len(IndexTable.Classification[IndexTable.index.get_loc('Size')].Items)
Nh = len(IndexTable.Classification[IndexTable.index.get_loc('Recycling_Process')].Items)
NS = len(IndexTable.Classification[IndexTable.index.get_loc('EV_penetration_scenario')].Items)
Na = len(IndexTable.Classification[IndexTable.index.get_loc('Chemistry_Scenarios')].Items)
Nz = len(IndexTable.Classification[IndexTable.index.get_loc('Stock_Scenarios')].Items)
NR = len(IndexTable.Classification[IndexTable.index.get_loc('Reuse_Scenarios')].Items)
NE = len(IndexTable.Classification[IndexTable.index.get_loc('Energy_Storage_Scenarios')].Items)
Nv = len(IndexTable.Classification[IndexTable.index.get_loc('V2G_Scenarios')].Items)

INFO (<ipython-input-1-fe30fc3b3d14> <<module>>): ### 1. - Initialize.
INFO (<ipython-input-1-fe30fc3b3d14> <<module>>): Read and parse config table, including the model index table, from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read parameter list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read process list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model run control from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model output control from model config sheet.
INFO (ODYM_Functions.py <ParseClassificationFile_Main>): End of file or formatting error while reading the classification file in column 16. Check if all classifications are present. If yes, you are good to go!
INFO (<ipython-input-1-fe30fc3b3d14> <<module>>): Define model classifications and select items for model classifications according to information provided by config file.
INFO (<ipython-input-1-fe30fc3b3d14> <<module>

Vehicle stock model for European fleet


## Preparing battery capacity
We define a simple battery capacity that depends on the drive train and size of the vehicle and is constant over time. This can be justified, since batteries are reaching a point where the range is no longer a limitation and improvements in battery technologies are targeted towards reducing the battery weight rather than increasing range. This is reflected in the material content of the different battery chemistries. 

We anyway use an array that depends on time in case we would like to change this assumption. 

In [2]:
CapArray = np.zeros((Ng,Ns,Nt))

In [3]:
# Defining empty DataFrame with the desired dimensions
lp0, lp1, lp2 = pd.core.reshape.util.cartesian_product([IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items,IndexTable.Classification[IndexTable.index.get_loc('Size')].Items, IndexTable.Classification[IndexTable.index.get_loc('Time')].Items])
df = pd.DataFrame(dict(Drive_train= lp0, Size=lp1, Time=lp2))

### Import data

In [4]:
# Defining path to raw data
data_path = os.path.join(os.getcwd(), 'data', 'raw_data')
# Importing data
data = pd.read_excel(data_path+'/Capacity.xlsx')
data.head()

,size,drive_train,required capacity,available capacity,energy_density module level,stated_specific_energy,Unnamed: 6,source
0,Small,BEV,33,33,0.277708,122,kWh/kg,"BatPack Model, Xu"
1,Medium,BEV,66,66,1.659485,384,kWh/kg,"BatPack Model, Xu"
2,Large,BEV,100,100,1.620221,384,kWh/kg,"BatPack Model, Xu"
3,Small,PHEV,17,17,1.170625,327,kWh/kg,"BatPack Model, Xu"
4,Medium,PHEV,8,8,0.147216,74,kWh/kg,"BatPack Model, Xu"


In [5]:
for g in (IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items):
    for s in (IndexTable.Classification[IndexTable.index.get_loc('Size')].Items):
        try: # We need this to ignore the drive trains that are not included
            df.loc[(df['Drive_train']==g) & (df['Size']==s), 'value'] = data.loc[(data['drive_train']==g) & (data['size']==s), 'available capacity'].values[0]
        except: 
            pass

In [6]:
df.fillna(0, inplace=True)

In [7]:
for m in range(0,len(df['Time'])):
    DriveTrainPosition = IndexTable.Classification[IndexTable.index.get_loc('Drive_train')].Items.index(df['Drive_train'].iloc[m])
    SizePosition = IndexTable.Classification[IndexTable.index.get_loc('Size')].Items.index(df['Size'].iloc[m])
    TimePosition = IndexTable.Classification[IndexTable.index.get_loc('Time')].Items.index(df['Time'].iloc[m])
    CapArray[DriveTrainPosition, SizePosition, TimePosition] = df['value'].iloc[m]

In [8]:
# define results path
results_path = os.path.join(os.getcwd(), 'data', 'scenario_data')

In [9]:
# Save as excel for overview
df.to_excel(results_path+'/capacity.xlsx')

In [10]:
np.save(results_path+'/capacity', CapArray, allow_pickle=True)